In [ ]:
from google.colab import drive

# 드라이브 마운트
drive.mount('/content/drive')

# 1. 파일이 저장된 정확한 경로 (본인의 경로에 맞게 수정하세요)
base_path = '/content/drive/MyDrive/ColabNotebooks/'

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 데이터 로드
training_data = pd.read_csv(f'{base_path}albumin_training_data.csv')

print(f"원본 데이터: {len(training_data):,}건")
print("\n데이터 컬럼:", training_data.columns.tolist())

원본 데이터: 506건

데이터 컬럼: ['AGE', 'GENDER', 'CKD', 'DIABETES', 'CCI', 'INITIAL_ALBUMIN', 'PROTEIN_INTAKE', 'DAYS_BETWEEN', 'ALBUMIN_CHANGE', 'FINAL_ALBUMIN']


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 전략 1: 데이터 품질 개선 (이상치 기준 완화)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("전략 1: 데이터 품질 재검토 (이상치 기준 완화)")
print("=" * 80)

# 필수 컬럼
required_cols = ['AGE', 'GENDER', 'CKD', 'DIABETES', 'CCI', 'INITIAL_ALBUMIN', 'PROTEIN_INTAKE',  'ALBUMIN_CHANGE']

# 결측치만 제거 (이상치 기준 대폭 완화)
data_clean = training_data[required_cols].dropna()

print(f"\n결측치 제거 후: {len(data_clean):,}건")

# 극단적 이상치만 제거 (기존보다 훨씬 완화)
print("\n이상치 제거 기준 (완화):")
print("  - Albumin 변화: ±5.0 g/dL 이내 (기존 ±3.0)")
print("  - 단백질 섭취: 300g/day 이하 (기존 200g)")
print("  - 초기 Albumin: 1.0~6.0 (기존 1.5~5.0)")

data_clean = data_clean[
    (data_clean['ALBUMIN_CHANGE'].abs() < 5.0) &
    (data_clean['PROTEIN_INTAKE'] < 300) &
    (data_clean['INITIAL_ALBUMIN'].between(1.0, 6.0))
]

print(f"이상치 제거 후: {len(data_clean):,}건")

# Target 분포 확인
print("\n📊 Target 변수 (ALBUMIN_CHANGE) 분포:")
print(data_clean['ALBUMIN_CHANGE'].describe())
print(f"\n양수 변화: {(data_clean['ALBUMIN_CHANGE'] > 0).sum():,}건 ({(data_clean['ALBUMIN_CHANGE'] > 0).mean()*100:.1f}%)")
print(f"음수 변화: {(data_clean['ALBUMIN_CHANGE'] < 0).sum():,}건 ({(data_clean['ALBUMIN_CHANGE'] < 0).mean()*100:.1f}%)")




전략 1: 데이터 품질 재검토 (이상치 기준 완화)

결측치 제거 후: 506건

이상치 제거 기준 (완화):
  - Albumin 변화: ±5.0 g/dL 이내 (기존 ±3.0)
  - 단백질 섭취: 300g/day 이하 (기존 200g)
  - 초기 Albumin: 1.0~6.0 (기존 1.5~5.0)
이상치 제거 후: 506건

📊 Target 변수 (ALBUMIN_CHANGE) 분포:
count    506.000000
mean      -0.259656
std        0.527943
min       -2.133333
25%       -0.581250
50%       -0.240833
75%        0.079545
max        1.500000
Name: ALBUMIN_CHANGE, dtype: float64

양수 변화: 146건 (28.9%)
음수 변화: 344건 (68.0%)


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 전략 2: Feature Engineering (강화)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("전략 2: Feature Engineering (대폭 강화)")
print("=" * 80)

# 기본 Feature
X = data_clean[['AGE', 'GENDER', 'CKD', 'DIABETES', 'INITIAL_ALBUMIN', 'PROTEIN_INTAKE']].copy()

# CCI 있으면 추가
if 'cci' in data_clean.columns:
    X['cci'] = data_clean['cci']

# 1) 단백질 섭취 관련 파생변수
X['protein_per_kg'] = X['PROTEIN_INTAKE'] / (X['AGE'] * 0.5 + 50)  # 대략적인 체중 추정
X['high_protein'] = (X['PROTEIN_INTAKE'] > 60).astype(int)
X['low_protein'] = (X['PROTEIN_INTAKE'] < 40).astype(int)

# 2) 초기 알부민 관련
X['low_baseline_albumin'] = (X['INITIAL_ALBUMIN'] < 3.5).astype(int)
X['very_low_baseline'] = (X['INITIAL_ALBUMIN'] < 3.0).astype(int)

# 3) 상호작용 변수 (임상적 의미)
X['CKD_protein'] = X['CKD'] * X['PROTEIN_INTAKE']
X['DIABETES_protein'] = X['DIABETES'] * X['PROTEIN_INTAKE']
X['baseline_protein'] = X['INITIAL_ALBUMIN'] * X['PROTEIN_INTAKE']
X['CKD_baseline'] = X['CKD'] * X['INITIAL_ALBUMIN']

# 4) 나이 관련
X['elderly'] = (X['AGE'] > 75).astype(int)
X['AGE_CKD'] = X['AGE'] * X['CKD']

# 5) 복합 위험도
X['high_risk'] = ((X['CKD'] == 1) | (X['DIABETES'] == 1) | (X['INITIAL_ALBUMIN'] < 3.0)).astype(int)
X['comorbidity_count'] = X['CKD'] + X['DIABETES']

# 6) 단백질 섭취의 비선형 효과
X['protein_squared'] = X['PROTEIN_INTAKE'] ** 2
X['protein_log'] = np.log1p(X['PROTEIN_INTAKE'])

# 7) 초기 알부민의 비선형 효과
X['albumin_squared'] = X['INITIAL_ALBUMIN'] ** 2
X['albumin_log'] = np.log(X['INITIAL_ALBUMIN'])

print(f"\n✅ 총 Feature 개수: {X.shape[1]}개")
print("\n생성된 Feature:")
for i, col in enumerate(X.columns, 1):
    print(f"   {i:2d}. {col}")

y = data_clean['ALBUMIN_CHANGE'].copy()

# Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"\nTrain set: {len(X_train):,}건")
print(f"Test set: {len(X_test):,}건")



전략 2: Feature Engineering (대폭 강화)

✅ 총 Feature 개수: 23개

생성된 Feature:
    1. AGE
    2. GENDER
    3. CKD
    4. DIABETES
    5. INITIAL_ALBUMIN
    6. PROTEIN_INTAKE
    7. protein_per_kg
    8. high_protein
    9. low_protein
   10. low_baseline_albumin
   11. very_low_baseline
   12. CKD_protein
   13. DIABETES_protein
   14. baseline_protein
   15. CKD_baseline
   16. elderly
   17. AGE_CKD
   18. high_risk
   19. comorbidity_count
   20. protein_squared
   21. protein_log
   22. albumin_squared
   23. albumin_log

Train set: 404건
Test set: 102건


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 전략 3: 여러 강력한 모델 시도
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("전략 3: 다양한 모델 비교")
print("=" * 80)

models = {}

# 1) XGBoost (성능이 가장 좋음)
print("\n⏳ XGBoost 학습 중...")
try:
    models['XGBoost'] = XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0.1,
        random_state=42,
        n_jobs=-1
    )
    models['XGBoost'].fit(X_train, y_train)
    print("✅ XGBoost 학습 완료")
except Exception as e:
    print(f"❌ XGBoost 설치 필요: pip install xgboost")
    print(f"   에러: {e}")

# 2) HistGradientBoosting (빠르고 강력)
print("\n⏳ HistGradientBoosting 학습 중...")
models['HistGBM'] = HistGradientBoostingRegressor(
    max_iter=200,
    learning_rate=0.05,
    max_depth=5,
    min_samples_leaf=20,
    l2_regularization=0.1,
    random_state=42
)
models['HistGBM'].fit(X_train, y_train)
print("✅ HistGBM 학습 완료")

# 3) Random Forest (기본)
print("\n⏳ Random Forest 학습 중...")
models['RandomForest'] = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)
models['RandomForest'].fit(X_train, y_train)
print("✅ RandomForest 학습 완료")

# 4) Gradient Boosting (원본)
print("\n⏳ Gradient Boosting 학습 중...")
models['GBM'] = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10,
    subsample=0.8,
    max_features='sqrt',
    random_state=42
)
models['GBM'].fit(X_train, y_train)
print("✅ GBM 학습 완료")



전략 3: 다양한 모델 비교

⏳ XGBoost 학습 중...
✅ XGBoost 학습 완료

⏳ HistGradientBoosting 학습 중...
✅ HistGBM 학습 완료

⏳ Random Forest 학습 중...
✅ RandomForest 학습 완료

⏳ Gradient Boosting 학습 중...
✅ GBM 학습 완료


In [ ]:

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 모델 평가 및 비교
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("📊 모델 성능 비교")
print("=" * 80)

results = []

for name, model in models.items():
    # 예측
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # 평가
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

    results.append({
        'Model': name,
        'Train R²': train_r2,
        'Test R²': test_r2,
        'Gap': train_r2 - test_r2,
        'Test MAE': test_mae,
        'Test RMSE': test_rmse,
        'CV R² (mean)': cv_scores.mean(),
        'CV R² (std)': cv_scores.std()
    })

    print(f"\n{name}:")
    print(f"   Train R²: {train_r2:.4f}, MAE: {train_mae:.4f}")
    print(f"   Test R²:  {test_r2:.4f}, MAE: {test_mae:.4f}")
    print(f"   CV R²:    {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    print(f"   Gap:      {train_r2 - test_r2:.4f}")

# 결과 정리
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Test R²', ascending=False)

print("\n" + "=" * 80)
print("📊 성능 순위 (Test R² 기준)")
print("=" * 80)
print("\n", results_df.to_string(index=False))



📊 모델 성능 비교

XGBoost:
   Train R²: 0.6187, MAE: 0.2587
   Test R²:  0.2229, MAE: 0.3548
   CV R²:    0.0905 (±0.1644)
   Gap:      0.3957

HistGBM:
   Train R²: 0.5085, MAE: 0.2935
   Test R²:  0.2610, MAE: 0.3406
   CV R²:    0.1074 (±0.1621)
   Gap:      0.2475

RandomForest:
   Train R²: 0.3764, MAE: 0.3281
   Test R²:  0.3335, MAE: 0.3163
   CV R²:    0.2428 (±0.1097)
   Gap:      0.0429

GBM:
   Train R²: 0.5470, MAE: 0.2798
   Test R²:  0.2415, MAE: 0.3477
   CV R²:    0.1251 (±0.1404)
   Gap:      0.3055

📊 성능 순위 (Test R² 기준)

        Model  Train R²  Test R²      Gap  Test MAE  Test RMSE  CV R² (mean)  CV R² (std)
RandomForest  0.376398 0.333495 0.042902  0.316309   0.425627      0.242763     0.109726
     HistGBM  0.508473 0.260958 0.247515  0.340633   0.448190      0.107446     0.162107
         GBM  0.547024 0.241515 0.305509  0.347686   0.454047      0.125114     0.140403
     XGBoost  0.618659 0.222937 0.395722  0.354778   0.459574      0.090463     0.164405


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 최고 성능 모델 선택 및 Feature Importance
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("🏆 최고 성능 모델")
print("=" * 80)

best_model_name = results_df.iloc[0]['Model']
best_test_r2 = results_df.iloc[0]['Test R²']

print(f"\n✅ {best_model_name}")
print(f"   Test R²: {best_test_r2:.4f}")

best_model = models[best_model_name]

# Feature Importance
print("\n" + "=" * 80)
print(f"Feature Importance ({best_model_name})")
print("=" * 80)

if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 15 중요 Feature:")
    for idx, row in importance_df.head(15).iterrows():
        bar = '█' * int(row['importance'] * 100)
        print(f"   {row['feature']:25s}: {row['importance']:.4f} {bar}")




🏆 최고 성능 모델

✅ RandomForest
   Test R²: 0.3335

Feature Importance (RandomForest)

Top 15 중요 Feature:
   INITIAL_ALBUMIN          : 0.2427 ████████████████████████
   albumin_squared          : 0.2311 ███████████████████████
   albumin_log              : 0.2261 ██████████████████████
   very_low_baseline        : 0.1031 ██████████
   low_baseline_albumin     : 0.0755 ███████
   AGE                      : 0.0382 ███
   high_risk                : 0.0200 █
   GENDER                   : 0.0149 █
   CKD_baseline             : 0.0125 █
   comorbidity_count        : 0.0098 
   DIABETES                 : 0.0097 
   AGE_CKD                  : 0.0083 
   elderly                  : 0.0044 
   CKD                      : 0.0038 
   PROTEIN_INTAKE           : 0.0000 


In [ ]:
best_model.feature_names_in_

array(['AGE', 'GENDER', 'CKD', 'DIABETES', 'INITIAL_ALBUMIN',
       'PROTEIN_INTAKE', 'protein_per_kg', 'high_protein', 'low_protein',
       'low_baseline_albumin', 'very_low_baseline', 'CKD_protein',
       'DIABETES_protein', 'baseline_protein', 'CKD_baseline', 'elderly',
       'AGE_CKD', 'high_risk', 'comorbidity_count', 'protein_squared',
       'protein_log', 'albumin_squared', 'albumin_log'], dtype=object)

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 전략 4: 이상치/극단값 환자 제거 후 재학습
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("전략 4: 예측 어려운 케이스 제외 후 재학습")
print("=" * 80)

# 첫 모델로 예측
y_train_pred = best_model.predict(X_train)
residuals = np.abs(y_train - y_train_pred)

# 상위 20% 큰 오차 케이스 제외
threshold = np.percentile(residuals, 80)
mask = residuals < threshold

X_train_filtered = X_train[mask]
y_train_filtered = y_train[mask]

print(f"\n원본 Train: {len(X_train):,}건")
print(f"필터링 후: {len(X_train_filtered):,}건 (제거: {(~mask).sum():,}건)")

# 재학습
print("\n⏳ 필터링 데이터로 재학습 중...")
if best_model_name == 'XGBoost':
    model_filtered = XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
elif best_model_name == 'HistGBM':
    model_filtered = HistGradientBoostingRegressor(
        max_iter=200,
        learning_rate=0.05,
        max_depth=5,
        min_samples_leaf=20,
        random_state=42
    )
else:
    model_filtered = GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        min_samples_split=20,
        min_samples_leaf=10,
        subsample=0.8,
        random_state=42
    )

model_filtered.fit(X_train_filtered, y_train_filtered)

y_train_pred = model_filtered.predict(X_train)
y_test_pred = model_filtered.predict(X_test)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"\n📊 필터링 모델 성능:")
print(f"   Train R²: {train_r2:.4f}")
print(f"   Test R²:  {test_r2:.4f}")
print(f"   Test MAE: {test_mae:.4f}")

if test_r2 > best_test_r2:
    print(f"\n✅ 성능 개선! ({best_test_r2:.4f} → {test_r2:.4f})")
    best_model = model_filtered
    best_test_r2 = test_r2
else:
    print(f"\n⚠️  성능 개선 없음")


전략 4: 예측 어려운 케이스 제외 후 재학습

원본 Train: 404건
필터링 후: 323건 (제거: 81건)

⏳ 필터링 데이터로 재학습 중...

📊 필터링 모델 성능:
   Train R²: 0.4385
   Test R²:  0.2684
   Test MAE: 0.3419

⚠️  성능 개선 없음


In [ ]:

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 최종 모델 저장
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("💾 최종 모델 저장")
print("=" * 80)

os.makedirs('models', exist_ok=True)

model_data = {
    'model': best_model,
    'feature_names': list(X.columns),
    'model_type': best_model_name,
    'test_r2': best_test_r2,
    'feature_engineering': True
}

model_path = 'models/albumin_predictor_improved.pkl'
joblib.dump(model_data, model_path)

print(f"\n✅ 저장 완료: {model_path}")
print(f"   모델: {best_model_name}")
print(f"   Test R²: {best_test_r2:.4f}")
print(f"   Feature 개수: {len(X.columns)}개")



💾 최종 모델 저장

✅ 저장 완료: models/albumin_predictor_improved.pkl
   모델: RandomForest
   Test R²: 0.3335
   Feature 개수: 23개


In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트 예측
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("🧪 테스트 예측")
print("=" * 80)

def prepare_patient_features(age, gender, ckd, diabetes, initial_albumin, protein_intake):
    """모델 feature 기준에 맞춘 입력 데이터 준비"""
    data = {
        # 기본 feature (대문자)
        'AGE': age,
        'GENDER': gender,
        'CKD': ckd,
        'DIABETES': diabetes,
        'INITIAL_ALBUMIN': initial_albumin,
        'PROTEIN_INTAKE': protein_intake,

        # 파생 feature
        'protein_per_kg': protein_intake / (age * 0.5 + 50),
        'high_protein': 1 if protein_intake > 60 else 0,
        'low_protein': 1 if protein_intake < 40 else 0,
        'low_baseline_albumin': 1 if initial_albumin < 3.5 else 0,
        'very_low_baseline': 1 if initial_albumin < 3.0 else 0,

        'CKD_protein': ckd * protein_intake,
        'DIABETES_protein': diabetes * protein_intake,
        'baseline_protein': initial_albumin * protein_intake,
        'CKD_baseline': ckd * initial_albumin,

        'elderly': 1 if age > 75 else 0,
        'AGE_CKD': age * ckd,

        'high_risk': 1 if (ckd == 1 or diabetes == 1 or initial_albumin < 3.0) else 0,
        'comorbidity_count': ckd + diabetes,

        'protein_squared': protein_intake ** 2,
        'protein_log': np.log1p(protein_intake),
        'albumin_squared': initial_albumin ** 2,
        'albumin_log': np.log(initial_albumin)
    }

    # 모델이 학습한 feature 순서로 정렬
    return pd.DataFrame([data])[best_model.feature_names_in_]


# 테스트 케이스
test_case = {
    'age': 75,
    'gender': 0,
    'ckd': 1,
    'diabetes': 0,
    'initial_albumin': 2.8,
    'protein_intake': 60
}

patient_X = prepare_patient_features(**test_case)
predicted_change = best_model.predict(patient_X)[0]
predicted_albumin = test_case['initial_albumin'] + predicted_change

print(f"\n환자 정보:")
print(f"   나이: {test_case['age']}세, 성별: {'여성' if test_case['gender'] == 0 else '남성'}")
print(f"   CKD: {'있음' if test_case['ckd'] == 1 else '없음'}")
print(f"   현재 Albumin: {test_case['initial_albumin']} g/dL")
print(f"   단백질 섭취: {test_case['protein_intake']} g/day")

print(f"\n📊 예측:")
print(f"   예상 변화: {predicted_change:+.2f} g/dL")
print(f"   4주 후 예상: {predicted_albumin:.2f} g/dL")


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 최종 요약
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("✅ 성능 개선 완료")
print("=" * 80)

print(f"""
📊 개선 결과:
   • 기존 Test R²: 0.1464
   • 개선 Test R²: {best_test_r2:.4f}
   • 개선폭: {best_test_r2 - 0.1464:+.4f}

🎯 최고 모델: {best_model_name}
📁 저장 위치: {model_path}
🔧 Feature 개수: {len(best_model.feature_names_in_)}개

💡 주요 개선 요소:
   1. Feature Engineering (20+ 파생 변수)
   2. 강력한 모델 (XGBoost / HistGBM)
   3. 이상치 기준 완화
   4. 비선형 관계 학습
""")



🧪 테스트 예측

환자 정보:
   나이: 75세, 성별: 여성
   CKD: 있음
   현재 Albumin: 2.8 g/dL
   단백질 섭취: 60 g/day

📊 예측:
   예상 변화: -0.04 g/dL
   4주 후 예상: 2.76 g/dL

✅ 성능 개선 완료

📊 개선 결과:
   • 기존 Test R²: 0.1464
   • 개선 Test R²: 0.3335
   • 개선폭: +0.1871

🎯 최고 모델: RandomForest
📁 저장 위치: models/albumin_predictor_improved.pkl
🔧 Feature 개수: 23개

💡 주요 개선 요소:
   1. Feature Engineering (20+ 파생 변수)
   2. 강력한 모델 (XGBoost / HistGBM)
   3. 이상치 기준 완화
   4. 비선형 관계 학습



# 성능 0.33에서 개선 방안들

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, RobustScaler
import warnings
warnings.filterwarnings('ignore')

try:
    from xgboost import XGBRegressor
    XGBOOST_AVAILABLE = True
except:
    XGBOOST_AVAILABLE = False
    print("⚠️  XGBoost 없음 (pip install xgboost)")

try:
    from lightgbm import LGBMRegressor
    LIGHTGBM_AVAILABLE = True
except:
    LIGHTGBM_AVAILABLE = False
    print("⚠️  LightGBM 없음 (pip install lightgbm)")

try:
    from catboost import CatBoostRegressor
    CATBOOST_AVAILABLE = True
except:
    CATBOOST_AVAILABLE = False
    print("⚠️  CatBoost 없음 (pip install catboost)")

# 경로 설정
base_path = '/content/drive/MyDrive/ColabNotebooks/'

print("=" * 80)
print("R² 추가 향상 전략 (0.33 → 0.40+ 목표)")
print("=" * 80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 데이터 로드 및 Feature Engineering
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

training_data = pd.read_csv(f'{base_path}albumin_training_data.csv')

# Fix: Changed column names to uppercase to match training_data DataFrame and added 'CCI'
required_cols = ['AGE', 'GENDER', 'CKD', 'DIABETES', 'CCI', 'INITIAL_ALBUMIN',
                'PROTEIN_INTAKE', 'ALBUMIN_CHANGE']

data_clean = training_data[required_cols].dropna()
data_clean = data_clean[
    (data_clean['ALBUMIN_CHANGE'].abs() < 5.0) &
    (data_clean['PROTEIN_INTAKE'] < 300) &
    (data_clean['INITIAL_ALBUMIN'].between(1.0, 6.0))
]

print(f"\n데이터: {len(data_clean):,}건")

# Feature Engineering (기존)
X = data_clean[['AGE', 'GENDER', 'CKD', 'DIABETES', 'INITIAL_ALBUMIN', 'PROTEIN_INTAKE']].copy()

# Fix: Changed 'cci' to 'CCI' if it exists in the original data
if 'CCI' in training_data.columns:
    X['CCI'] = data_clean['CCI']

# 기존 파생 변수
X['protein_per_kg'] = X['PROTEIN_INTAKE'] / (X['AGE'] * 0.5 + 50) # Fix: Corrected casing
X['high_protein'] = (X['PROTEIN_INTAKE'] > 60).astype(int) # Fix: Corrected casing
X['low_protein'] = (X['PROTEIN_INTAKE'] < 40).astype(int) # Fix: Corrected casing
X['low_baseline_albumin'] = (X['INITIAL_ALBUMIN'] < 3.5).astype(int) # Fix: Corrected casing
X['very_low_baseline'] = (X['INITIAL_ALBUMIN'] < 3.0).astype(int) # Fix: Corrected casing
X['ckd_protein'] = X['CKD'] * X['PROTEIN_INTAKE'] # Fix: Corrected casing
X['diabetes_protein'] = X['DIABETES'] * X['PROTEIN_INTAKE'] # Fix: Corrected casing
X['baseline_protein'] = X['INITIAL_ALBUMIN'] * X['PROTEIN_INTAKE'] # Fix: Corrected casing
X['ckd_baseline'] = X['CKD'] * X['INITIAL_ALBUMIN'] # Fix: Corrected casing
X['elderly'] = (X['AGE'] > 75).astype(int) # Fix: Corrected casing
X['age_ckd'] = X['AGE'] * X['CKD'] # Fix: Corrected casing
X['high_risk'] = ((X['CKD'] == 1) | (X['DIABETES'] == 1) | (X['INITIAL_ALBUMIN'] < 3.0)).astype(int) # Fix: Corrected casing
X['comorbidity_count'] = X['CKD'] + X['DIABETES'] # Fix: Corrected casing
X['protein_squared'] = X['PROTEIN_INTAKE'] ** 2 # Fix: Corrected casing
X['protein_log'] = np.log1p(X['PROTEIN_INTAKE']) # Fix: Corrected casing
X['albumin_squared'] = X['INITIAL_ALBUMIN'] ** 2 # Fix: Corrected casing
X['albumin_log'] = np.log(X['INITIAL_ALBUMIN']) # Fix: Corrected casing

y = data_clean['ALBUMIN_CHANGE'].copy() # Fix: Changed 'albumin_change' to 'ALBUMIN_CHANGE'

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"Train: {len(X_train):,}건, Test: {len(X_test):,}건")



# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 전략 2: 최신 Boosting 라이브러리 시도
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("전략 2: 최신 Boosting 모델")
print("=" * 80)

models = {}

# 1) LightGBM (빠르고 성능 좋음)
if LIGHTGBM_AVAILABLE:
    print("\n⏳ LightGBM 학습 중...")
    models['LightGBM'] = LGBMRegressor(
        n_estimators=500,
        learning_rate=0.03,
        max_depth=7,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=42,
        verbose=-1
    )
    models['LightGBM'].fit(X_train, y_train) # Fix: Changed X_train_enh to X_train
    print("✅ LightGBM 완료")

# 2) CatBoost (범주형 변수 강함)
if CATBOOST_AVAILABLE:
    print("\n⏳ CatBoost 학습 중...")
    models['CatBoost'] = CatBoostRegressor(
        iterations=500,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        random_seed=42,
        verbose=0
    )
    models['CatBoost'].fit(X_train, y_train) # Fix: Changed X_train_enh to X_train
    print("✅ CatBoost 완료")

# 3) XGBoost (튜닝)
if XGBOOST_AVAILABLE:
    print("\n⏳ XGBoost (튜닝) 학습 중...")
    models['XGBoost_Tuned'] = XGBRegressor(
        n_estimators=500,
        learning_rate=0.03,
        max_depth=6,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0.1,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1
    )
    models['XGBoost_Tuned'].fit(X_train, y_train) # Fix: Changed X_train_enh to X_train
    print("✅ XGBoost 완료")

# 4) HistGBM (튜닝)
print("\n⏳ HistGradientBoosting (튜닝) 학습 중...")
models['HistGBM_Tuned'] = HistGradientBoostingRegressor(
    max_iter=500,
    learning_rate=0.03,
    max_depth=7,
    min_samples_leaf=15,
    l2_regularization=1.0,
    random_state=42
)
models['HistGBM_Tuned'].fit(X_train, y_train) # Fix: Changed X_train_enh to X_train
print("✅ HistGBM 완료")


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 전략 3: 앙상블 (Voting & Stacking)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("전략 3: 앙상블 모델")
print("=" * 80)

if len(models) >= 2:
    # Voting Ensemble
    print("\n⏳ Voting Ensemble 학습 중...")
    voting_models = [(name, model) for name, model in list(models.items())[:3]]
    models['Voting'] = VotingRegressor(estimators=voting_models)
    models['Voting'].fit(X_train, y_train) # Fix: Changed X_train_enh to X_train
    print("✅ Voting 완료")

    # Stacking Ensemble
    print("\n⏳ Stacking Ensemble 학습 중...")
    base_models = list(models.items())[:3]
    models['Stacking'] = StackingRegressor(
        estimators=base_models,
        final_estimator=Ridge(alpha=1.0),
        cv=5
    )
    models['Stacking'].fit(X_train, y_train) # Fix: Changed X_train_enh to X_train
    print("✅ Stacking 완료")


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 전략 4: Neural Network
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("전략 4: Neural Network")
print("=" * 80)

# 데이터 스케일링 (NN에 필수)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train) # Fix: Changed X_train_enh to X_train
X_test_scaled = scaler.transform(X_test) # Fix: Changed X_test_enh to X_test

print("\n⏳ Neural Network 학습 중...")
models['NeuralNet'] = MLPRegressor(
    hidden_layer_sizes=(128, 64, 32),
    activation='relu',
    solver='adam',
    alpha=0.01,
    learning_rate='adaptive',
    max_iter=500,
    early_stopping=True,
    validation_fraction=0.2,
    random_state=42
)
models['NeuralNet'].fit(X_train_scaled, y_train)
print("✅ Neural Network 완료")


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 전략 5: 이상치 제거 후 재학습 (더 강화)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("전략 5: 이상치 제거 강화")
print("=" * 80)

# 최고 성능 모델로 예측
best_temp_model = list(models.values())[0]
if 'NeuralNet' in models:
    y_train_pred_temp = models['NeuralNet'].predict(X_train_scaled)
else:
    y_train_pred_temp = best_temp_model.predict(X_train) # Fix: Changed X_train_enh to X_train

residuals = np.abs(y_train - y_train_pred_temp)

# 상위 15% 제거 (기존 20%보다 강화)
threshold = np.percentile(residuals, 85)
mask = residuals < threshold

X_train_filtered = X_train[mask] # Fix: Changed X_train_enh to X_train
y_train_filtered = y_train[mask]

print(f"\n원본: {len(X_train):,}건") # Fix: Changed X_train_enh to X_train
print(f"필터링 후: {len(X_train_filtered):,}건 (제거: {(~mask).sum():,}건)")

# 재학습
if LIGHTGBM_AVAILABLE:
    print("\n⏳ LightGBM (필터링 데이터) 학습 중...")
    models['LightGBM_Filtered'] = LGBMRegressor(
        n_estimators=500,
        learning_rate=0.03,
        max_depth=7,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=42,
        verbose=-1
    )
    models['LightGBM_Filtered'].fit(X_train_filtered, y_train_filtered)
    print("✅ 완료")


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 모든 모델 평가
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("📊 모든 모델 성능 비교")
print("=" * 80)

results = []

for name, model in models.items():
    # 예측 (NN은 스케일링 필요)
    if name == 'NeuralNet':
        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)
    else:
        y_train_pred = model.predict(X_train) # Fix: Changed X_train_enh to X_train
        y_test_pred = model.predict(X_test) # Fix: Changed X_test_enh to X_test

    # 평가
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    results.append({
        'Model': name,
        'Train R²': train_r2,
        'Test R²': test_r2,
        'Gap': train_r2 - test_r2,
        'Test MAE': test_mae,
        'Test RMSE': test_rmse
    })

    print(f"\n{name}:")
    print(f"   Train R²: {train_r2:.4f}")
    print(f"   Test R²:  {test_r2:.4f}")
    print(f"   Test MAE: {test_mae:.4f}")
    print(f"   Gap:      {train_r2 - test_r2:.4f}")

# 결과 정리
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Test R²', ascending=False)

print("\n" + "=" * 80)
print("🏆 최종 순위")
print("=" * 80)
print("\n", results_df.to_string(index=False))


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 최고 모델 저장
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("💾 최고 성능 모델 저장")
print("=" * 80)

best_model_name = results_df.iloc[0]['Model']
best_test_r2 = results_df.iloc[0]['Test R²']
best_model = models[best_model_name]

print(f"\n🏆 {best_model_name}")
print(f"   Test R²: {best_test_r2:.4f}")
print(f"   개선: 0.3335 → {best_test_r2:.4f} ({best_test_r2 - 0.3335:+.4f})")

os.makedirs('models', exist_ok=True)

model_data = {
    'model': best_model,
    'feature_names': list(X.columns), # Fix: Changed X_enhanced.columns to X.columns
    'model_type': best_model_name,
    'test_r2': best_test_r2,
    'scaler': scaler if best_model_name == 'NeuralNet' else None
}

model_path = 'models/albumin_predictor_best.pkl'
joblib.dump(model_data, model_path)

print(f"\n✅ 저장 완료: {model_path}")


# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 최종 요약
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n" + "=" * 80)
print("✅ 성능 개선 완료")
print("=" * 80)

print(f"""
📊 최종 결과:
   • 기존 R²: 0.3335 (RandomForest)
   • 최고 R²: {best_test_r2:.4f} ({best_model_name})
   • 개선폭: {best_test_r2 - 0.3335:+.4f}

🔧 적용 전략:
   ✅ Feature Engineering 강화 (23개 → {X.shape[1]}개) # Fix: Changed X_enhanced.shape[1] to X.shape[1]
   ✅ 최신 Boosting (LightGBM/CatBoost)
   ✅ 앙상블 (Voting/Stacking)
   ✅ Neural Network
   ✅ 이상치 제거 강화

💡 추가 개선 방법:
   1. 더 많은 데이터 수집
   2. 추가 Feature (BMI, 약물 정보 등)
   3. 시계열 데이터 활용 (이전 변화 추이)
   4. Domain-specific Feature (영양학 전문가 자문)
""")

# Top 3 모델 비교
print("\n📊 Top 3 모델:")
for i, row in results_df.head(3).iterrows():
    print(f"   {row['Model']:20s}: R² {row['Test R²']:.4f}, MAE {row['Test MAE']:.4f}")

⚠️  CatBoost 없음 (pip install catboost)
R² 추가 향상 전략 (0.33 → 0.40+ 목표)

데이터: 506건
Train: 404건, Test: 102건

전략 2: 최신 Boosting 모델

⏳ LightGBM 학습 중...
✅ LightGBM 완료

⏳ XGBoost (튜닝) 학습 중...
✅ XGBoost 완료

⏳ HistGradientBoosting (튜닝) 학습 중...
✅ HistGBM 완료

전략 3: 앙상블 모델

⏳ Voting Ensemble 학습 중...
✅ Voting 완료

⏳ Stacking Ensemble 학습 중...
✅ Stacking 완료

전략 4: Neural Network

⏳ Neural Network 학습 중...
✅ Neural Network 완료

전략 5: 이상치 제거 강화

원본: 404건
필터링 후: 343건 (제거: 61건)

⏳ LightGBM (필터링 데이터) 학습 중...
✅ 완료

📊 모든 모델 성능 비교

LightGBM:
   Train R²: 0.6153
   Test R²:  0.2188
   Test MAE: 0.3565
   Gap:      0.3965

XGBoost_Tuned:
   Train R²: 0.7085
   Test R²:  0.1994
   Test MAE: 0.3620
   Gap:      0.5091

HistGBM_Tuned:
   Train R²: 0.6823
   Test R²:  0.1421
   Test MAE: 0.3783
   Gap:      0.5402

Voting:
   Train R²: 0.6779
   Test R²:  0.2004
   Test MAE: 0.3637
   Gap:      0.4775

Stacking:
   Train R²: 0.5335
   Test R²:  0.2368
   Test MAE: 0.3446
   Gap:      0.2967

NeuralNet:
   Train R²: 0.

📊 개선 결과:
   • 기존 Test R²: 0.1464
   • 개선 Test R²: 0.3335
   • 개선폭: +0.1871

🎯 최고 모델: RandomForest

📁 저장 위치: models/albumin_predictor_improved.pkl

**-> 최종 albumin_predictor_improved.pkl**